In [92]:
import PyPDF2
import camelot
import pandas as pd

In [93]:
#FAILED PYPDF IMPLEMENTATION

# text = ""
# with open("engg-credit-transfer-database.pdf", 'rb') as file:
#     reader = PyPDF2.PdfReader(file)
#     for page in reader.pages:
#         text += page.extract_text()

# with open("tester.txt", "w+") as tester_file:
#     tester_file.write(text)


In [94]:
engg_file = open("engg-credit-transfer-database.pdf", 'rb')

#store data in pdfReader
pdfReader = PyPDF2.PdfReader(engg_file)

#count number of pages
totalPages = len(pdfReader.pages)
totalPages

39

In [71]:
engg_pdf = camelot.read_pdf("engg-credit-transfer-database.pdf", 
                       pages='1-end') #address of file location
initial_df = pd.DataFrame()

#add data to DF
for i in range(totalPages + 1):
    if i == 0: initial_df = engg_pdf[i].df
    else: initial_df = pd.concat([initial_df, engg_pdf[i].df], ignore_index=True)


In [95]:
initial_df.columns = ["BEng_Prog",
                    "Host_Institution",
                    "Country",
                    "Course_Code_Host",
                    "Course_Title_Host",
                    "Course_Code_HKU",
                    "Course_Title_HKU",
                    "Remarks"]

In [96]:
#cleaning if it is null, blank, or if it's a header
cleaned_df = initial_df[initial_df['Host_Institution'].str.strip().ne('') & 
                        (initial_df['Host_Institution'] != 'Host_Institution') & (initial_df['Host_Institution'].notna())]

#elective/discipline elective distinction
def set_course_code(row):
    if row['Course_Code_HKU'] == '':
        if 'discipline' in row['Course_Title_HKU'].lower():
            return row['BEng_Prog'] + 'XXXX'
        else:
            return 'ELECTIVE'
    return row['Course_Code_HKU']  # Return the existing value if not blank

# apply the function
cleaned_df['Course_Code_HKU'] = cleaned_df.apply(set_course_code, axis=1)

# encoding issues
cleaned_df['Host_Institution'] = cleaned_df['Host_Institution'].str.replace(
    "Queen‚Äôs University, Kingston", 
    "Queen's University, Kingston", 
    regex=False
)

cleaned_df[:5]

/var/folders/nt/67mjll3s4fb1vnp8mvn0q2kh0000gn/T/ipykernel_8874/1694360878.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Course_Code_HKU'] = cleaned_df.apply(set_course_code, axis=1)
/var/folders/nt/67mjll3s4fb1vnp8mvn0q2kh0000gn/T/ipykernel_8874/1694360878.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Host_Institution'] = cleaned_df['Host_Institution'].str.replace(


,BEng_Prog,Host_Institution,Country,Course_Code_Host,Course_Title_Host,Course_Code_HKU,Course_Title_HKU,Remarks
6,ME,Griffith University,Australia,2201ENG,Engineering Thermodynamics,MECH3402,Engineering thermodynamics,NaN
7,ME,Griffith University,Australia,3004ENG,Project Management Principles,MECH4410,Engineering & technology management,NaN
8,ME,Griffith University,Australia,3319ENG,Power Transmission and Distribution,MECH3406,Electrical and electronic engineering,NaN
9,ME,Griffith University,Australia,3505ENG,Heat and Mass Transfer,MECH4411,Heat transfer,NaN
10,ME,Griffith University,Australia,3511ENG,Design of Machine Elements,MECH3427,Design and manufacture,NaN


In [97]:
# FOR CHECKING PURPOSES
# cleaned_df.iloc[716]
cleaned_df.to_csv("engg-credit_transfer_database.csv",index=False)